# Part 1 -- Setup & Collection

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [1]:
!pwd

/home/jovyan/work/predicting_stock_market_trends_with_Twitter/ipynb


In [2]:
from os import chdir
chdir('/home/jovyan/work/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

In [3]:
!pip install pymongo pyquery tweepy tqdm
import pymongo
import tweepy
# from tqdm import tqdm 

In [4]:
# Replace the API_KEY and API_SECRET with your application's key and secret
auth = tweepy.AppAuthHandler(my_keys['CONSUMER_KEY'], my_keys['CONSUMER_SECRET'])

# Authorize twitter, initialize tweepy
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [5]:
cli = pymongo.MongoClient(host='35.163.253.8')

In [37]:
# Instantiates when you put data in 
status_collection = cli.twitter_db.status_collection
complete_collection = cli.twitter_db.complete_collection
test_collection = cli.twitter_db.test_collection
tweets_collection = cli.twitter_db.tweets_collection
cli.database_names()

[u'twitter_db']

In [58]:
print(status_collection.count())
print(complete_collection.count())
print(test_collection.count())
print(tweets_collection.count())

0
0
0
0


In [57]:
status_collection.drop()
complete_collection.drop()
test_collection.drop()
tweets_collection.drop()

In [9]:
chdir('//home/jovyan/work/predicting_stock_market_trends_with_Twitter/tools/GetOldTweets-python-master')
!pip install -r requirements.txt
import got

  Using cached lxml-3.5.0.tar.gz
  Using cached pyquery-1.2.10-py2-none-any.whl
  Running setup.py bdist_wheel for lxml ... - \ | / error
  Complete output from command /opt/conda/envs/python2/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-6IHx1i/lxml/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpHNbuXhpip-wheel- --python-tag cp27:
  Building lxml version 3.5.0.
  Building without Cython.
  ERROR: /bin/sh: 1: xslt-config: not found
  
  ** make sure the development packages of libxml2 and libxslt are installed **
  
  Using build configuration of libxslt
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-2.7
  creating build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/__init__.py -> build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/usedoctest.py -> build/lib.linux-x86_64-2.7/lxml
  

### Implement distributed processing from scratch - run this on multiple AWS instances:

In [10]:
twitter_30_df = pd.read_csv('/home/jovyan/work/predicting_stock_market_trends_with_Twitter/data/twitter_users_30.csv')
twitter_30_df.sample(3)

,Name,Handle,Followers,Description,Type,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
19,Downtown Josh Brown,ReformedBroker,"663,000","Chairman of the Twitter Federal Reserve, star ...",Thought Leader,NaN,NaN,NaN,NaN
27,Mark Yusko,MarkYusko,"18,700",Founder & Chief Investment Officer Morgan Cree...,Thought Leader,NaN,NaN,NaN,NaN
21,CNN Tech,cnntech,"538,000","The latest in tech news and trends, focusing o...",News Outlet,NaN,NaN,NaN,NaN


In [50]:
handle = twitter_30_df['Handle']

In [ ]:
# for i, h in enumerate(handle):
#     status_collection.insert_one({'handle': h})

In [59]:
def test_get_tweets_to_mongo(handle_lookup):
    tweet_list = []
    for i in handle_lookup:
        try:
            tweetCriteria = got.manager.TweetCriteria().setUsername(i).setMaxTweets(50)
            tweet = got.manager.TweetManager.getTweets(tweetCriteria)
            tweetCount = 0
            for t in tweet:
                tweet_dict = {
                    'username': t.username,
                    'text': t.text,
                    'timestamp': t.date
                }
                tweet_list.append(tweet_dict)
                tweetCount += 1
                if (len(tweet_list)+1)%10==True:
                    print("Downloaded {} tweets".format(len(tweet_list)))
                    
                try:
                    mongo_response = test_collection.insert_one(tweet_dict)
                except:
                    pass
            print("Inserted {0} tweets to Mongo for user: {1}".format(tweetCount, i))
        except:
            pass
    print ("\nTotal tweets downloaded & stored in mongo: {}".format(len(tweet_list)))

In [60]:
from datetime import datetime
start = datetime.now()
test_get_tweets_to_mongo(handle)
end = datetime.now()
print(end - start)

Inserted 4 tweets to Mongo for user: BarackObama
Downloaded 10 tweets
Downloaded 20 tweets
Downloaded 30 tweets
Downloaded 40 tweets
Downloaded 50 tweets
Inserted 50 tweets to Mongo for user: cnnbrk
Downloaded 60 tweets
Inserted 8 tweets to Mongo for user: BillGates
Downloaded 70 tweets
Downloaded 80 tweets
Downloaded 90 tweets
Downloaded 100 tweets
Inserted 47 tweets to Mongo for user: realDonaldTrump
Downloaded 110 tweets
Downloaded 120 tweets
Downloaded 130 tweets
Downloaded 140 tweets
Downloaded 150 tweets
Inserted 50 tweets to Mongo for user: TheEconomist
Downloaded 160 tweets
Inserted 9 tweets to Mongo for user: aplusk
Downloaded 170 tweets
Inserted 7 tweets to Mongo for user: HillaryClinton
Downloaded 180 tweets
Downloaded 190 tweets
Downloaded 200 tweets
Downloaded 210 tweets
Downloaded 220 tweets
Inserted 50 tweets to Mongo for user: TechCrunch
Downloaded 230 tweets
Downloaded 240 tweets
Inserted 22 tweets to Mongo for user: elonmusk
Downloaded 250 tweets
Downloaded 260 tweets

In [40]:
#this works!
def get_tweets_to_mongo(handle_lookup):
    tweet_list = []
    for i in handle_lookup:
        try:
            tweetCriteria = got.manager.TweetCriteria().setUsername(i).setMaxTweets(6000)
            tweet = got.manager.TweetManager.getTweets(tweetCriteria)
            tweetCount = 0
            for t in tweet:
                tweet_dict = {
                    'username': t.username,
                    'text': t.text,
                    'timestamp': t.date
                }
                tweet_list.append(tweet_dict)
                tweetCount += 1
                try:    
                    mongo_response = tweets_collection.insert_one(tweet_dict)
                except:
                    pass
        except:
            pass

    print ("\nTotal tweets downloaded & stored in mongo: {}".format(len(tweet_list)))

In [41]:
from datetime import datetime
start = datetime.now()
get_tweets_to_mongo(handle)
end = datetime.now()
print(end - start)


Total tweets downloaded & stored in mongo: 8130
0:03:11.176625


In [61]:
test_collection.count()

1083

In [62]:
tweets_collection.count()

0

In [64]:
curs = test_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

,_id,text,timestamp,username
0,5931c27c307e01022d602b51,"On this National Gun Violence Awareness Day, l...",2017-06-02 17:35:54,BarackObama
1,5931c27c307e01022d602b52,Forever grateful for the service and sacrifice...,2017-05-29 13:09:16,BarackObama
2,5931c27c307e01022d602b53,Good to see my friend Prince Harry in London t...,2017-05-27 13:15:25,BarackObama
3,5931c27c307e01022d602b54,"Through faith, love, and resolve the character...",2017-05-25 14:13:35,BarackObama
4,5931c27d307e01022d602b55,Ex-Penn State president sentenced to jail for ...,2017-06-02 19:33:02,cnnbrk
5,5931c27d307e01022d602b56,Man arrested wearing wedding band stolen from ...,2017-06-02 19:25:04,cnnbrk
6,5931c27d307e01022d602b57,"Ireland's ruling party elects Leo Varadkar, th...",2017-06-02 17:50:06,cnnbrk
7,5931c27d307e01022d602b58,ISIS says it was behind deadly Philippines cas...,2017-06-02 13:05:58,cnnbrk
8,5931c27d307e01022d602b59,34 bodies were found inside the Manila casino ...,2017-06-02 04:11:19,cnnbrk
9,5931c27d307e01022d602b5a,The Trump administration Thursday night asked ...,2017-06-02 03:11:54,cnnbrk


In [ ]:
df.shape